<a href="https://colab.research.google.com/github/Sreekar-Kandhadai/pyspark-interview-questions/blob/main/Twitter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
We have a table of employees that includes the following fields: id, first_name, last_name, age, sex, employee_title,
department, salary, target, bonus, city, address, and manager_id. We need to find the top 3 distinct salaries for
each department. The output should include:

The department name.
The top 3 distinct salaries for each department.
The results should be ordered alphabetically by department and then by the highest salary to the lowest salary.

In [3]:
from pyspark.sql import SparkSession

from pyspark.sql.functions import *

from pyspark.sql.window import Window

spark=SparkSession.builder.appName("top_3_salaries").getOrCreate()

data = [
    (1, 'Allen', 'Wang', 55, 'F', 'Manager', 'Management', 200000, 0, 300, 'California', '23St', 1),
    (13, 'Katty', 'Bond', 56, 'F', 'Manager', 'Management', 150000, 0, 300, 'Arizona', None, 1),
    (19, 'George', 'Joe', 50, 'M', 'Manager', 'Management', 100000, 0, 300, 'Florida', '26St', 1),
    (11, 'Richerd', 'Gear', 57, 'M', 'Manager', 'Management', 250000, 0, 300, 'Alabama', None, 1),
    (10, 'Jennifer', 'Dion', 34, 'F', 'Sales', 'Sales', 100000, 200, 150, 'Alabama', None, 13),
    (18, 'Laila', 'Mark', 26, 'F', 'Sales', 'Sales', 100000, 200, 150, 'Florida', '23St', 11),
    (20, 'Sarrah', 'Bicky', 31, 'F', 'Senior Sales', 'Sales', 200000, 200, 150, 'Florida', '53St', 19),
    (21, 'Suzan', 'Lee', 34, 'F', 'Sales', 'Sales', 130000, 200, 150, 'Florida', '56St', 19),
    (22, 'Mandy', 'John', 31, 'F', 'Sales', 'Sales', 130000, 200, 150, 'Florida', '45St', 19),
    (17, 'Mick', 'Berry', 44, 'M', 'Senior Sales', 'Sales', 220000, 200, 150, 'Florida', None, 11),
    (12, 'Shandler', 'Bing', 23, 'M', 'Auditor', 'Audit', 110000, 200, 150, 'Arizona', None, 11),
    (14, 'Jason', 'Tom', 23, 'M', 'Auditor', 'Audit', 100000, 200, 150, 'Arizona', None, 11),
    (16, 'Celine', 'Anston', 27, 'F', 'Auditor', 'Audit', 100000, 200, 150, 'Colorado', None, 11),
    (15, 'Michale', 'Jackson', 44, 'F', 'Auditor', 'Audit', 70000, 150, 150, 'Colorado', None, 11),
    (6, 'Molly', 'Sam', 28, 'F', 'Sales', 'Sales', 140000, 100, 150, 'Arizona', '24St', 13),
    (7, 'Nicky', 'Bat', 33, 'F', 'Sales', 'Sales', None, None, None, None, None, None)
]

columns = ["id", "first_name", "last_name", "age", "sex", "employee_title", "department", "salary",
           "target", "bonus", "city", "address", "manager_id"]


df=spark.createDataFrame(data,columns)

df.show()

window_spec=Window.partitionBy('department').orderBy(col('salary').desc())

df1=df.withColumn("rank",dense_rank().over(window_spec))

df2=df1.filter("rank <=3").select("department","salary").distinct().orderBy(col('department'),col('salary').desc())

df2.show()

+---+----------+---------+---+---+--------------+----------+------+------+-----+----------+-------+----------+
| id|first_name|last_name|age|sex|employee_title|department|salary|target|bonus|      city|address|manager_id|
+---+----------+---------+---+---+--------------+----------+------+------+-----+----------+-------+----------+
|  1|     Allen|     Wang| 55|  F|       Manager|Management|200000|     0|  300|California|   23St|         1|
| 13|     Katty|     Bond| 56|  F|       Manager|Management|150000|     0|  300|   Arizona|   NULL|         1|
| 19|    George|      Joe| 50|  M|       Manager|Management|100000|     0|  300|   Florida|   26St|         1|
| 11|   Richerd|     Gear| 57|  M|       Manager|Management|250000|     0|  300|   Alabama|   NULL|         1|
| 10|  Jennifer|     Dion| 34|  F|         Sales|     Sales|100000|   200|  150|   Alabama|   NULL|        13|
| 18|     Laila|     Mark| 26|  F|         Sales|     Sales|100000|   200|  150|   Florida|   23St|        11|
|